In [ ]:
###########################################################################################################################################################
# Citations:
# 1) Divyanshupy (2019). MobileNet-Object-Detection [Repository]. GitHub. https://github.com/Divyanshupy/MobileNet-Object-Detection
###########################################################################################################################################################

In [ ]:
############################################
# Import the relevant libraries and set the working directory appropriately
############################################

In [ ]:
from PIL import Image
import numpy as np
import os
import numpy as np
import pandas as pd
from natsort import natsorted
import re
import os
import os
import random
import pandas as pd
import torch
import numpy as np
import random
# from skimage import io
import skimage
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn as nn
import torch.nn.functional as F
import os
import datetime
import sys
import torch as t
import torch.cuda as cuda
import torch.optim as optimizer
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import torchvision.transforms as transf
import numpy as np
import matplotlib.pyplot as plt
from skimage import io, transform
from scipy import io
from torchsummary import summary
import csv
import os
from google.colab import drive
drive.mount('/content/gdrive')

# Set the scenario number
scenario_numx = 5
scenario_num = scenario_numx

all_cars = True

# Set the working directory appropriately in this variable
directory = '/content/gdrive/MyDrive/Dataset/scenario' + str(scenario_numx)
directory = directory + '/'

! git clone https://github.com/Divyanshupy/MobileNet-Object-Detection.git

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
fatal: destination path 'MobileNet-Object-Detection' already exists and is not an empty directory.


In [ ]:
pip install torchinfo

In [ ]:
directory = '/content/gdrive/MyDrive/Dataset/scenario' + str(scenario_numx) + '/resources'
directory = directory + '/'

In [ ]:
########################################################################################
# Intialize the filnames of the images in a list
########################################################################################

In [ ]:
import os
import glob


path = directory + 'camera_data'
filenames = os.listdir(path)
original_list = [os.path.join(path, filename) for filename in filenames]
full_path_filenames = natsorted(original_list)



files = os.listdir(path)
files = natsorted(files)


In [ ]:
########################################################################################
# Generate bounding boxes of cars and store the bounding box coordinates in a dataframe
# Divyanshupy (2019). MobileNet-Object-Detection [Repository]. GitHub. https://github.com/Divyanshupy/MobileNet-Object-Detection
########################################################################################
import numpy as np
import argparse
import cv2

import matplotlib.pyplot as plt
import pandas as pd




column_names = ['image name','label','startX', 'startY', 'endX', 'endY']
df = pd.DataFrame(columns=column_names)
for _,j in enumerate(full_path_filenames):
  args = my_dict = {"image": j, "prototxt": "/content/MobileNet-Object-Detection/MobileNetSSD_deploy.prototxt.txt" , "model":"/content/MobileNet-Object-Detection/MobileNetSSD_deploy.caffemodel", "confidence":0.40 }
  CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
	"bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
	"dog", "horse", "motorbike", "person", "pottedplant", "sheep",
	"sofa", "train", "tvmonitor"]
  COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))


  net = cv2.dnn.readNetFromCaffe(args["prototxt"], args["model"])

  image = cv2.imread(args["image"])
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

  (h, w) = image.shape[:2]
  blob = cv2.dnn.blobFromImage(cv2.resize(image, (h, w)), 0.007843, (h, w), 127.5)


  net.setInput(blob)
  detections = net.forward()


  for i in np.arange(0, detections.shape[2]):

    confidence = detections[0, 0, i, 2]

    if confidence > args["confidence"]:
      idx = int(detections[0, 0, i, 1])
      box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
      (startX, startY, endX, endY) = box.astype("int")

      # display the prediction
      label = "{}: {:.2f}%".format(CLASSES[idx], confidence * 100)
      # print("[INFO] {}".format(label))
      cv2.rectangle(image, (startX, startY), (endX, endY),
        COLORS[idx], 2)
      y = startY - 15 if startY - 15 > 15 else startY + 15
      cv2.putText(image, label, (startX, y),
        cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[idx], 2)

      if startX<0:
        startX = 0

      if startY<0:
        staryY =0

      if endX >960:
        endX=960

      if endY>540:
        endY=540  

      # print(label)
      if label[0:3] =='car':
        new_row = pd.DataFrame([[ files[_],label,startX, startY, endX, endY]], columns=['image name','label','startX', 'startY', 'endX', 'endY'])
        df = df.append(new_row, ignore_index=True)

  # plt.imshow(image)
  # plt.show()
  cv2.waitKey(1)

In [ ]:
############################################
# Save the pandas dataframe as a csv file
############################################

In [ ]:
df2= df
column = 'label'
substrings = ['car', 'bus']
df2 = df2.loc[df2[column].str.contains('|'.join(substrings))]
df2.to_csv('scenario5_bbox_mobilenet.csv', index=False, sep='\t', encoding='utf-16')